In [1]:
import pandas as pd
import numpy as np
from keras.layers import Dense, Flatten, LSTM, GRU, SimpleRNN, Embedding
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import re

In [2]:
df = pd.read_csv('../datasets/IMDB Dataset.csv')

In [3]:
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [5]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [6]:
df['review'] = df['review'].apply(lambda x: re.sub(r'[,.!&]', '', x))

In [7]:
df['review']

0        One of the other reviewers has mentioned that ...
1        A wonderful little production <br /><br />The ...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot bad dialogue bad acting idiotic direc...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

In [8]:
tokenizer = Tokenizer()

In [9]:
input_sequences = np.array(df['review'])

In [10]:
input_sequences[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked They are right as this is exactly what happened with me<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence which set in right from the word GO Trust me this is not a show for the faint hearted or timid This show pulls no punches with regards to drugs sex or violence Its is hardcore in the classic use of the word<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary It focuses mainly on Emerald City an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda Em City is home to manyAryans Muslims gangstas Latinos Christians Italians Irish and moreso scuffles death stares dodgy dealings and shady agreements are never far away<br /><br />I would say the main appeal of the show is due to the fact that it goes where other sh

In [11]:
train_sequences = tokenizer.fit_on_texts(input_sequences)

In [13]:
tokenizer.fit_on_texts(input_sequences)

In [14]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'are': 23,
 'his': 24,
 'have': 25,
 'be': 26,
 'one': 27,
 'he': 28,
 'at': 29,
 'all': 30,
 'by': 31,
 'an': 32,
 'they': 33,
 'so': 34,
 'who': 35,
 'from': 36,
 'like': 37,
 'or': 38,
 'just': 39,
 'her': 40,
 'about': 41,
 'out': 42,
 'if': 43,
 "it's": 44,
 'has': 45,
 'some': 46,
 'there': 47,
 'what': 48,
 'good': 49,
 'more': 50,
 'when': 51,
 'very': 52,
 'up': 53,
 'no': 54,
 'time': 55,
 'my': 56,
 'even': 57,
 'would': 58,
 'she': 59,
 'which': 60,
 'only': 61,
 'really': 62,
 'see': 63,
 'story': 64,
 'their': 65,
 'had': 66,
 'can': 67,
 'me': 68,
 'were': 69,
 'well': 70,
 'than': 71,
 'much': 72,
 'we': 73,
 'been': 74,
 'get': 75,
 'bad': 76,
 'do': 77,
 'other': 78,
 'will': 79,
 'great': 80,
 'also': 81,
 'into': 82,
 'p

In [15]:
vocab_size = len(tokenizer.word_counts) + 1
embeded_dim = 64
max_length = 140

In [16]:
vocab_size

143780

In [17]:
train_sequences = tokenizer.texts_to_sequences(input_sequences)

In [18]:
padded_sequences = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')

In [19]:
padded_sequences.shape

(50000, 140)

In [20]:
y = np.array(df['review'].apply(lambda x: 1 if x == 'positive' else 0))

In [21]:
y.shape

(50000,)

In [22]:
from keras.models import Sequential

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train ,X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.2, random_state=41)

In [28]:
# Normal Embedding Word Model
model1 = Sequential()

model1.add(Embedding(input_dim=vocab_size, output_dim=embeded_dim, input_length=max_length))
model1.add(Flatten())
model1.add(Dense(units=20, activation='relu'))
model1.add(Dense(units=1, activation='sigmoid'))

In [29]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [30]:
history_model1 = model1.fit(X_train, y_train, epochs=5)

Epoch 1/5
1250/1250 [==============================] - 95s 76ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 2/5
1250/1250 [==============================] - 92s 73ms/step - loss: 3.0883e-06 - accuracy: 1.0000
Epoch 3/5
1250/1250 [==============================] - 92s 73ms/step - loss: 1.0116e-06 - accuracy: 1.0000
Epoch 4/5
1250/1250 [==============================] - 94s 75ms/step - loss: 4.2892e-07 - accuracy: 1.0000
Epoch 5/5
1250/1250 [==============================] - 93s 74ms/step - loss: 2.0181e-07 - accuracy: 1.0000


In [32]:
model1.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.6062e-07 - accuracy: 1.0000


[1.6061919438925543e-07, 1.0]

In [33]:
# Simple RNN Model
model2 = Sequential()

model2.add(Embedding(input_dim=vocab_size, output_dim = embeded_dim, input_length=max_length))
model2.add(SimpleRNN(units=64))
model2.add(Dense(units=10, activation='relu'))
model2.add(Dense(units=1, activation='sigmoid'))

In [34]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'mse'])

In [36]:
model2.fit(X_train, y_train, epochs=4)

Epoch 1/4
1250/1250 [==============================] - 97s 78ms/step - loss: 1.1046e-05 - accuracy: 1.0000 - mse: 1.4194e-10
Epoch 2/4
1250/1250 [==============================] - 97s 78ms/step - loss: 3.4366e-06 - accuracy: 1.0000 - mse: 1.2739e-11
Epoch 3/4
1250/1250 [==============================] - 97s 78ms/step - loss: 1.4178e-06 - accuracy: 1.0000 - mse: 2.1230e-12
Epoch 4/4
1250/1250 [==============================] - 97s 78ms/step - loss: 6.5229e-07 - accuracy: 1.0000 - mse: 4.4562e-13


In [37]:
model2.evaluate(X_test, y_test)

313/313 [==============================] - 2s 7ms/step - loss: 4.4189e-07 - accuracy: 1.0000 - mse: 1.9560e-13


[4.418904495651077e-07, 1.0, 1.9559631963140395e-13]